In [1]:
from ftplib import FTP

#Connect to FTP Storage
ftp = FTP('ftp.boxgrad.com')
ftp.login(user='data@idxexchange.com', passwd='Real_estate123$')

# List what’s in the root directory
ftp.retrlines('LIST')


drwxr-xr-x   2 data@idxexchange.com 1002         4096 Mar 11  2025 bridgeoutput
drwxr-xr-x   2 data@idxexchange.com 1002         4096 Mar 11  2025 crmls
-rw-r--r--   1 data@idxexchange.com 1002            0 Sep 22 13:35 etc
drwxr-xr-x   3 data@idxexchange.com 1002         4096 Jun  3 17:04 raw
drwxr-xr-x   4 data@idxexchange.com 1002         4096 Jun  3 17:04 resources
drwxr-xr-x   4 data@idxexchange.com 1002         4096 Jun  1 10:20 tableau


'226 Transfer complete'

In [2]:
ftp.cwd('raw')        # move inside 'raw'
ftp.retrlines('LIST')

drwxr-xr-x   2 data@idxexchange.com 1002         4096 Sep  9 19:12 california


'226 Transfer complete'

In [3]:
ftp.cwd('california')
ftp.retrlines('LIST')

-rw-r--r--   1 data@idxexchange.com 1002     85737472 Jun  8 20:53 CRMLSSold20220101_20231231_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002      9781769 Jun 13  2024 CRMLSSold202401_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     10869834 Jun 13  2024 CRMLSSold202402_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     12726365 Jun 13  2024 CRMLSSold202403_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     13476537 Jun 13  2024 CRMLSSold202404_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     14916967 Jun 13  2024 CRMLSSold202405_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     13716857 Jul 12  2024 CRMLSSold202406_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     14802093 Sep 18  2024 CRMLSSold202407_filled.csv
-rw-r--r--   1 data@idxexchange.com 1002     13708354 Sep 18  2024 CRMLSSold202408.csv
-rw-r--r--   1 data@idxexchange.com 1002     11846407 Oct  8  2024 CRMLSSold202409.csv
-rw-r--r--   1 data@idxexchange.com 1002     12967887 Nov  4  2

'226 Transfer complete'

In [4]:
files = ftp.nlst()  # list all files
files = [f for f in files if f.startswith("CRMLSSold")]
files.sort()  # order by name (so months are sorted)

# Download the latest 13 months (Aug 2024 → Aug 2025)
files_to_download = files[-13:]  # last 13 monthly files
print("Downloading these files:", files_to_download)

for filename in files_to_download:
    with open(filename, 'wb') as f:
        ftp.retrbinary('RETR ' + filename, f.write)
        print("Downloaded:", filename)


Downloaded: CRMLSSold202408.csv
Downloaded: CRMLSSold202409.csv
Downloaded: CRMLSSold202410.csv
Downloaded: CRMLSSold202411.csv
Downloaded: CRMLSSold202412.csv
Downloaded: CRMLSSold202501_filled.csv
Downloaded: CRMLSSold202502.csv
Downloaded: CRMLSSold202503.csv
Downloaded: CRMLSSold202504.csv
Downloaded: CRMLSSold202505.csv
Downloaded: CRMLSSold202506.csv
Downloaded: CRMLSSold202507.csv
Downloaded: CRMLSSold202508.csv


In [5]:
ftp.quit()

'221 Goodbye.'

In [6]:
import pandas as pd

# Example: load one file
df = pd.read_csv("CRMLSSold202503.csv")
print(df.head())
print(df.info())

         BuyerAgentAOR         ListAgentAOR Flooring ViewYN WaterfrontYN  \
0             SanDiego             SanDiego      NaN  False          NaN   
1            Southland            Southland      NaN   True          NaN   
2          PacificWest          PacificWest      NaN   True          NaN   
3             SanDiego             SanDiego      NaN  False          NaN   
4  NorthSanDiegoCounty  NorthSanDiegoCounty      NaN   True          NaN   

  BasementYN PoolPrivateYN  OriginalListPrice  ListingKey  \
0        NaN         False             3500.0   551930077   
1        NaN           NaN           300000.0   537152404   
2        NaN           NaN            65000.0   537067171   
3        NaN         False             3200.0   522929639   
4        NaN           NaN          8800000.0   521707294   

                  ListAgentEmail  ...            LotSizeDimensions  \
0  claudette@claudettecooper.com  ...                          NaN   
1        avlandman@sbcglobal.net  ..

These are the most meaningful, based on real estate logic:

LivingArea → square feet of the house

BedroomsTotal → total bedrooms

BathroomsTotalInteger → total bathrooms

LotSizeSquareFeet → total land/lot size in sqft

Stories → number of floors

GarageSpaces → how many cars fit in the garage

YearBuilt → year house was built

PostalCode → location proxy (or use Latitude + Longitude)

Latitude → exact location (north/south)

Longitude → exact location (east/west)

PoolPrivateYN → does it have a pool? (True/False)

NewConstructionYN → is it new construction? (True/False)

Year from CloseDate

Month from Close

In [7]:
file_list = files_to_download
df_all = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)

print("Combined shape:", df_all.shape)

/tmp/ipython-input-3391570818.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)


Combined shape: (284421, 80)


In [8]:
df_all["CloseDate"] = pd.to_datetime(df_all["CloseDate"], errors="coerce")
print(df_all["CloseDate"].min(), "→", df_all["CloseDate"].max())


2024-08-01 00:00:00 → 2025-08-31 00:00:00


In [9]:
#Keep only Residential + SingleFamilyResidence rows:
df_all = df_all[
    (df_all["PropertyType"] == "Residential") &
    (df_all["PropertySubType"] == "SingleFamilyResidence")
]
print("After filtering:", df_all.shape)

After filtering: (143536, 80)


In [10]:
#Extract Year and Month from the CloseDate:
df_all["CloseDate"] = pd.to_datetime(df_all["CloseDate"], errors="coerce")
df_all["Year"] = df_all["CloseDate"].dt.year
df_all["Month"] = df_all["CloseDate"].dt.month

In [11]:
keep_cols = [
    "ClosePrice",          # target
    "LivingArea",
    "BedroomsTotal",
    "BathroomsTotalInteger",
    "LotSizeSquareFeet",
    "Stories",
    "GarageSpaces",
    "YearBuilt",
    "PostalCode",
    "Latitude",
    "Longitude",
    "PoolPrivateYN",
    "NewConstructionYN",
    "Year",
    "Month"
]
df_model = df_all[keep_cols]
print(df_model.head())

    ClosePrice  LivingArea  BedroomsTotal  BathroomsTotalInteger  \
22    275000.0      1175.0            3.0                    2.0   
23   1363500.0       920.0            2.0                    2.0   
24    650000.0      1381.0            3.0                    2.0   
25   1000000.0      1198.0            3.0                    2.0   
26  29550000.0      3728.0            5.0                    6.0   

    LotSizeSquareFeet  Stories  GarageSpaces  YearBuilt PostalCode   Latitude  \
22            26136.0      1.0          10.0     1965.0      95965  39.514633   
23             3074.0      1.0           0.0     1919.0      90245  33.924152   
24             6100.0      1.0           2.0     1968.0      93033  34.159717   
25             6200.0      1.0           2.0     1960.0      92648  33.697311   
26             5626.0      2.0           2.0     2017.0      92630  33.671772   

     Longitude PoolPrivateYN NewConstructionYN  Year  Month  
22 -121.602277         False             F

### Data Cleaning

In [12]:
# Checking missing values only in the selected columns
df_model = df_all[[
    "ClosePrice","LivingArea","BedroomsTotal","BathroomsTotalInteger",
    "LotSizeSquareFeet","Stories","GarageSpaces","YearBuilt",
    "PostalCode","Latitude","Longitude","PoolPrivateYN","NewConstructionYN",
    "Year","Month"
]]

df_model.isna().sum().sort_values(ascending=False)


,0
Stories,18495
PoolPrivateYN,13865
NewConstructionYN,10328
GarageSpaces,5458
LotSizeSquareFeet,2471
YearBuilt,118
LivingArea,77
BathroomsTotalInteger,26
Latitude,19
Longitude,19


In [13]:
# PoolPrivateYN → map True/False to 1/0, fill missing with 0
df_model.loc[:, "PoolPrivateYN"] = df_model["PoolPrivateYN"].map({"True": 1, "False": 0})
df_model.loc[:, "PoolPrivateYN"] = df_model["PoolPrivateYN"].fillna(0)

# NewConstructionYN → map True/False to 1/0, fill missing with 0
df_model.loc[:, "NewConstructionYN"] = df_model["NewConstructionYN"].map({"True": 1, "False": 0})
df_model.loc[:, "NewConstructionYN"] = df_model["NewConstructionYN"].fillna(0)

# GarageSpaces → fill missing with 0
df_model.loc[:, "GarageSpaces"] = df_model["GarageSpaces"].fillna(0)


/tmp/ipython-input-1370017927.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model.loc[:, "PoolPrivateYN"] = df_model["PoolPrivateYN"].fillna(0)
/tmp/ipython-input-1370017927.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model.loc[:, "NewConstructionYN"] = df_model["NewConstructionYN"].fillna(0)


In [14]:
# Check missing values only for the columns we just fixed
print(df_model[["PoolPrivateYN", "NewConstructionYN", "GarageSpaces"]].isna().sum())


PoolPrivateYN        0
NewConstructionYN    0
GarageSpaces         0
dtype: int64


In [15]:
df_model.isna().sum().sort_values(ascending=False)


,0
Stories,18495
LotSizeSquareFeet,2471
YearBuilt,118
LivingArea,77
BathroomsTotalInteger,26
Latitude,19
Longitude,19
ClosePrice,1
PostalCode,1
GarageSpaces,0


In [16]:
df_model[["LivingArea","LotSizeSquareFeet","Stories","BathroomsTotalInteger","YearBuilt"]].describe()


,LivingArea,LotSizeSquareFeet,Stories,BathroomsTotalInteger,YearBuilt
count,143459.000000,1.410650e+05,125041.000000,143510.000000,143418.000000
mean,2029.604246,1.935474e+05,1.352556,2.613644,1975.355625
std,1041.628658,1.215918e+07,0.477768,1.200381,27.488442
min,0.000000,0.000000e+00,1.000000,0.000000,1800.000000
25%,1378.000000,5.663000e+03,1.000000,2.000000,1955.000000
50%,1800.000000,7.250000e+03,1.000000,2.000000,1976.000000
75%,2413.000000,1.030900e+04,2.000000,3.000000,1997.000000
max,56500.000000,2.087221e+09,2.000000,153.000000,2026.000000


In [17]:
# List of numeric columns to fix
num_cols = [
    "LivingArea",
    "BedroomsTotal",
    "BathroomsTotalInteger",
    "LotSizeSquareFeet",
    "Stories",
    "YearBuilt"
]

#Fill missing values with each columns median
for col in num_cols:
  median_val = df_model[col].median()
  df_model.loc[:,col] = df_model[col].fillna(median_val)
  print(f"{col}: filled NaNs with median = {median_val}")

LivingArea: filled NaNs with median = 1800.0
BedroomsTotal: filled NaNs with median = 3.0
BathroomsTotalInteger: filled NaNs with median = 2.0
LotSizeSquareFeet: filled NaNs with median = 7250.0
Stories: filled NaNs with median = 1.0
YearBuilt: filled NaNs with median = 1976.0


In [18]:
df_model.isna().sum().sort_values(ascending=False)

,0
Longitude,19
Latitude,19
ClosePrice,1
PostalCode,1
BathroomsTotalInteger,0
LivingArea,0
BedroomsTotal,0
GarageSpaces,0
Stories,0
LotSizeSquareFeet,0


In [19]:
df_model = df_model.dropna(subset=["Latitude", "Longitude", "PostalCode"])


In [20]:
from ftplib import FTP

# reconnect
ftp = FTP('ftp.boxgrad.com')
ftp.login(user='data@idxexchange.com', passwd='Real_estate123$')

ftp.cwd('raw/california')
files = ftp.nlst()
files = [f for f in files if f.startswith("CRMLSSold")]
files.sort()

print("Available CRMLS files:", files[-12:])  # show last 12 months
ftp.quit()


Available CRMLS files: ['CRMLSSold202409.csv', 'CRMLSSold202410.csv', 'CRMLSSold202411.csv', 'CRMLSSold202412.csv', 'CRMLSSold202501_filled.csv', 'CRMLSSold202502.csv', 'CRMLSSold202503.csv', 'CRMLSSold202504.csv', 'CRMLSSold202505.csv', 'CRMLSSold202506.csv', 'CRMLSSold202507.csv', 'CRMLSSold202508.csv']


'221 Goodbye.'

In [23]:
# Count how many rows have missing ClosePrice in each Year-Month group
missing_close_summary = (
    df_model[df_model["ClosePrice"].isna()]
    .groupby(["Year", "Month"])
    .size()
    .reset_index(name="MissingCount")
    .sort_values(["Year", "Month"])
)

print(missing_close_summary)


   Year  Month  MissingCount
0  2024     12             1


In [24]:
df_model = df_model.dropna(subset=["ClosePrice"])


In [25]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error

# --------------------------------------------------------------------
# 1) Create a boolean mask for the test set (August 2025)
# --------------------------------------------------------------------
is_test_set_mask = (df_model["Year"] == 2025) & (df_model["Month"] == 8)

# Apply mask to get test set DataFrame
test_data_df = df_model[is_test_set_mask]

# Train set = everything before August 2025
train_data_df = df_model[~is_test_set_mask &
                         ((df_model["Year"] < 2025) |
                          ((df_model["Year"] == 2025) & (df_model["Month"] < 8)))]

print("Train set shape:", train_data_df.shape)
print("Test set shape :", test_data_df.shape)

# --------------------------------------------------------------------
# 2) Define features (X) and target (y)
# --------------------------------------------------------------------
selected_features_list = [
    "LivingArea", "BedroomsTotal", "BathroomsTotalInteger",
    "LotSizeSquareFeet", "Stories", "GarageSpaces",
    "YearBuilt", "Latitude", "Longitude",
    "PoolPrivateYN", "NewConstructionYN"
]

X_train_df = train_data_df[selected_features_list]
y_train_series = train_data_df["ClosePrice"]

X_test_df = test_data_df[selected_features_list]
y_test_series = test_data_df["ClosePrice"]

# --------------------------------------------------------------------
# 3) Train Linear Regression model
# --------------------------------------------------------------------
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_df, y_train_series)

# --------------------------------------------------------------------
# 4) Predictions on test data
# --------------------------------------------------------------------
y_predicted_array = linear_regression_model.predict(X_test_df)

# --------------------------------------------------------------------
# 5) Evaluation metrics
# --------------------------------------------------------------------
r2_score_value = r2_score(y_test_series, y_predicted_array)

mape_value = mean_absolute_percentage_error(y_test_series, y_predicted_array)
median_mape_value = np.median(np.abs((y_test_series - y_predicted_array) / y_test_series))
#R^2 is how much of that spread (variance) in house prices does the model explain using the features.
print(f"R²: {r2_score_value:.4f}")
print(f"MAPE: {mape_value*100:.2f}%")
print(f"Median MAPE: {median_mape_value*100:.2f}%")


Train set shape: (132062, 15)
Test set shape : (11453, 15)
R²: 0.4655
MAPE: 53.33%
Median MAPE: 35.60%


R² (0.45): The model explains about 45% of the variation in house prices.

MAPE (57%): On average, predictions are off by ~57% of the true sale price.

Median MAPE (39%): For a typical home, predictions are off by ~39%,a few luxury or unusual homes (like mansions, huge estates, or very cheap fixer-uppers) cause big errors that make the overall average error (MAPE) look worse than what most typical homes experience.

In [26]:
print("Rows:", df_model.shape[0])
print("Columns:", df_model.shape[1])

Rows: 143515
Columns: 15


In [27]:
len(df_model)

143515

In [28]:
from sklearn.tree import DecisionTreeRegressor

# Initialize model
decision_tree_model = DecisionTreeRegressor(
    max_depth=10,   # controls how complex the tree is
    random_state=42
)

# Train on same data
decision_tree_model.fit(X_train_df, y_train_series)

# Predict
y_pred_dt = decision_tree_model.predict(X_test_df)

# Evaluate
r2_dt = r2_score(y_test_series, y_pred_dt)
mape_dt = mean_absolute_percentage_error(y_test_series, y_pred_dt)
median_mape_dt = np.median(np.abs((y_test_series - y_pred_dt) / y_test_series))

print("\n--- Decision Tree ---")
print(f"R²: {r2_dt:.4f}")
print(f"MAPE: {mape_dt*100:.2f}%")
print(f"Median MAPE: {median_mape_dt*100:.2f}%")



--- Decision Tree ---
R²: -0.2464
MAPE: 27.88%
Median MAPE: 17.71%


In [29]:
from sklearn.ensemble import RandomForestRegressor

random_forest_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

random_forest_model.fit(X_train_df, y_train_series)
y_pred_rf = random_forest_model.predict(X_test_df)

r2_rf = r2_score(y_test_series, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test_series, y_pred_rf)
median_mape_rf = np.median(np.abs((y_test_series - y_pred_rf) / y_test_series))

print("\n--- Random Forest ---")
print(f"R²: {r2_rf:.4f}")
print(f"MAPE: {mape_rf*100:.2f}%")
print(f"Median MAPE: {median_mape_rf*100:.2f}%")



--- Random Forest ---
R²: -0.5173
MAPE: 18.14%
Median MAPE: 9.92%
